<a href="https://colab.research.google.com/github/SeongminCC/nongnet_AI/blob/main/modeling_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# validation AUC, MSE, RMSE, Accuracy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import LSTM   # LSTM 층 
from sklearn.model_selection import train_test_split
import random
import os

import matplotlib.pylab as plt
import seaborn as sns


from sklearn.preprocessing import MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error, r2_score, roc_auc_score

import os
import numpy as np

import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import time

from torchvision import transforms

import copy
from torchvision.utils import save_image
from torch.utils.tensorboard import SummaryWriter

import math


# 경고 끄기
warnings.filterwarnings(action='ignore')

# 시드고정
tf.random.set_seed(19970119)
random.seed(19970119)
np.random.seed(19970119)



%cd /content/drive/MyDrive/DNA/농넷_AIFactory/aT_data
!ls -al

df1 = pd.read_csv('./data/train/train_0.csv')


df2 = df1.fillna(method='ffill') # 결측치 위치 기준 윗 값 가져오기 
df2 = df1.fillna(method='bfill') # 결측치 위치 기준 아랫 값 가져오기


date = pd.to_datetime(df1['datadate'], format='%Y%m%d')
df2['datadate'] = date 


df3 = df2.iloc[:, :28]

scaler = MinMaxScaler()
scale_cols = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체평균가격(원)', '해당일자_전체거래물량(kg)',
              '하위가격 평균가(원)', '상위가격 평균가(원)', '하위가격 거래물량(kg)', '상위가격 거래물량(kg)', '일자별_도매가격_최대(원)',
                '일자별_도매가격_평균(원)', '일자별_도매가격_최소(원)', '일자별_소매가격_최대(원)', '일자별_소매가격_평균(원)',
                '일자별_소매가격_최소(원)', '수출중량(kg)', '수출금액(달러)', '수입중량(kg)', '수입금액(달러)',
                '무역수지(달러)', '주산지_0_초기온도(℃)', '주산지_0_최대온도(℃)', '주산지_0_최저온도(℃)']
                
df_scaled = scaler.fit_transform(df3[scale_cols])

df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols


TEST_SIZE = 14  # 2주간 데이터 예측

train = df_scaled[:-TEST_SIZE]
test = df_scaled[-TEST_SIZE:]


def make_dataset(data, label, window_size=4):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)


feature_cols = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체거래물량(kg)',
                '하위가격 평균가(원)', '상위가격 평균가(원)', '하위가격 거래물량(kg)', '상위가격 거래물량(kg)', '일자별_도매가격_최대(원)',
                '일자별_도매가격_평균(원)', '일자별_도매가격_최소(원)', '일자별_소매가격_최대(원)', '일자별_소매가격_평균(원)',
                '일자별_소매가격_최소(원)', '수출중량(kg)', '수출금액(달러)', '수입중량(kg)', '수입금액(달러)',
                '무역수지(달러)', '주산지_0_초기온도(℃)', '주산지_0_최대온도(℃)', '주산지_0_최저온도(℃)']
label_cols = ['해당일자_전체평균가격(원)']

train_feature = train[feature_cols]
train_label = train[label_cols]

test_feature = test[feature_cols]
test_label = test[label_cols]

# train dataset
train_feature, train_label = make_dataset(train_feature, train_label)

# train, validation set 생성
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)
# ((1154, 4, 26), (289, 4, 26))

# test dataset (실제 예측 해볼 데이터)
test_feature, test_label = make_dataset(test_feature, test_label)
# ((10, 4, 26), (10, 1))






######################################################################################################################
clf = RandomForestRegressor()

test_size = 14
X_train = df_scaled.drop(columns = '해당일자_전체평균가격(원)').iloc[:-test_size]
y_train = df_scaled['해당일자_전체평균가격(원)'].iloc[:-test_size]

X_test = df_scaled.drop(columns = '해당일자_전체평균가격(원)').iloc[-test_size:]
y_test = df_scaled['해당일자_전체평균가격(원)'].iloc[-test_size:]

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

MSE = mean_squared_error(y_test, y_pred)
AUC = roc_auc_score(y_test, y_pred)
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
Accuracy = clf.score(y_test, y_pred)

print(MSE)
print(AUC)
print(RMSE)
print(Accuracy)














/content/drive/MyDrive/DNA/농넷_AIFactory/aT_data
total 5270
-rw------- 1 root root   21765 Aug 28 08:31 answer_example.csv
-rw------- 1 root root 4737091 Sep  5 06:04 aT_베이스라인.ipynb
drwx------ 2 root root    4096 Sep  4 13:59 aT_test_raw
drwx------ 2 root root    4096 Sep  4 13:59 aT_train_raw
drwx------ 3 root root    4096 Sep  4 13:59 data
-rw------- 1 root root  591766 Sep 12 13:47 modeling_2weeks.ipynb
-rw------- 1 root root   16530 Sep 14 05:14 modeling_matlab.ipynb
-rw------- 1 root root   15743 Aug 28 12:05 preprocessing.py


ValueError: ignored

In [ ]:
writer_train = SummaryWriter(log_dir='/content/drive/MyDrive/DNA/농넷_AIFactory/aT_data/data/Log/AUC')
writer_train = SummaryWriter(log_dir='/content/drive/MyDrive/DNA/농넷_AIFactory/aT_data/data/Log/Accuracy')
writer_train = SummaryWriter(log_dir='/content/drive/MyDrive/DNA/농넷_AIFactory/aT_data/data/Log/MSE')
writer_train = SummaryWriter(log_dir='/content/drive/MyDrive/DNA/농넷_AIFactory/aT_data/data/Log/RMSE')

writer_train.add_scalar("MSE", loss, epoch)